## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-09-52-55 +0000,bbc,Plan for electricity bills based on region dro...,https://www.bbc.com/news/articles/cdr3e78112po
1,2025-07-10-09-51-52 +0000,bbc,Royal Mail to scrap second-class post on Satur...,https://www.bbc.com/news/articles/c36x8k2612ko
2,2025-07-10-09-51-10 +0000,bbc,US cuts visa validity for applicants from thre...,https://www.bbc.com/news/articles/cx23ywg9w82o
3,2025-07-10-09-35-06 +0000,nyt,UK and France Agree to First Ever Nuclear Weap...,https://www.nytimes.com/2025/07/10/world/europ...
4,2025-07-10-09-34-46 +0000,nyt,U.S. Imposes Sanctions on U.N. Special Rapport...,https://www.nytimes.com/2025/07/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,17
45,russia,6
47,ukraine,5
37,tariff,5
105,attack,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
70,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,42
46,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...,40
89,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,39
32,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,39
33,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,34


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
70,42,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
85,25,2025-07-09-13-21-24 +0000,nyt,European Court Holds Russia Liable for Human R...,https://www.nytimes.com/2025/07/09/world/europ...
89,22,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
3,19,2025-07-10-09-35-06 +0000,nyt,UK and France Agree to First Ever Nuclear Weap...,https://www.nytimes.com/2025/07/10/world/europ...
32,19,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...
83,19,2025-07-09-13-51-41 +0000,bbc,French police raid on National Rally HQ prompt...,https://www.bbc.com/news/articles/ckg5kd04e1jo
34,19,2025-07-10-00-36-34 +0000,bbc,"US sanctions UN expert Francesca Albanese, cri...",https://www.bbc.com/news/articles/c70rllxr0kyo
61,17,2025-07-09-19-04-54 +0000,bbc,Watch: Timelapse shows floodwaters swamping Ne...,https://www.bbc.com/news/videos/cx23ydrvvrwo
21,15,2025-07-10-05-11-58 +0000,bbc,North Sea operators 'running out of time' to p...,https://www.bbc.com/news/articles/cn4101grdxvo
58,15,2025-07-09-19-47-41 +0000,nyt,Can Canada Leverage Its Unique U.S. Relationsh...,https://www.nytimes.com/2025/07/09/world/canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
